In [1]:
import requests
import os.path
import pickle
import PyPDF2
import re 

In [2]:
states = {  "Andhra_Pradesh" : "AP",
            "Arunachal_Pradesh" : "AR", 
            "Assam" : "AS",
            "Bihar" : "BR", 
            "Chhattisgarh" : "CT",
            "Goa": "GA",
            "Gujarat" : "GJ",
            "Haryana" : "HR",
            "Himachal_Pradesh" : "HP",
            "Jharkhand" : "JH", 
            "Karnataka" : "KA",
            "Kerala" : "KL",
            "Madhya_Pradesh" : "MP",
            "Maharashtra" : "MH",
            "Manipur" : "MN",
            "Meghalaya" : "ML",
            "Mizoram" : "MZ",
            "Nagaland" : "NL",
            "Odisha" : "OR",
            "Punjab" : "PB",
            "Rajasthan" : "RJ",
            "Sikkim" : "SK",
            "Tamil_Nadu" : "TN",
            "Telangana" : "TG",
            "Tripura" : "TR",
            "Uttar_Pradesh" : "UP",
            "Uttarakhand" : "UT",
            "West_Bengal" : "WB",
            "Andaman_Nicobar_Islands" : "AN",
            "Chandigarh" : "CH",
            "Dadra_Nagar_Haveli_Daman_Diu" : "DD",
            "NCT_Delhi": "DL",
            "Jammu_Kashmir" : "JK",
            "Ladakh" : "LH",
            "Lakshadweep": "LD",
            "Puducherry" : "PY"
            }

1. Download State and UT Data (PDFs)

In [3]:
for key in states:
    #print(key)
    path_to_file = states[key]+".pdf"
    url = "http://rchiips.org/nfhs/NFHS-5_FCTS/COMPENDIUM/" + key + ".pdf"
    if os.path.exists(path_to_file)==False and int(requests.get(url, stream=True).headers['Content-length']) > 0:
        response = requests.get(url)
        if response.status_code == requests.codes.ok:
            file = open(states[key]+".pdf", "wb")
            file.write(response.content)
            file.close()

2. Read State and UT files to obtain names of districts

In [4]:
def getdistricts(statename):
    try: 
        pdfFileObj = open(states[statename]+".pdf", 'rb')    
    except: 
        print(states[statename]+".pdf: No such file found\n")
        return []
    else:  
        print("PDF for the state of "+states[statename]+" exists")
        pdfReader  = PyPDF2.PdfFileReader(pdfFileObj)
        pageObj    = pdfReader.getPage(2)
        pagetext   = pageObj.extractText()
        pagetext  = pagetext.strip()
        if 'Key Indicator' not in pagetext or 'District' not in pagetext:
            return []
        pageObj2    = pdfReader.getPage(3)
        pagetext2   = pageObj2.extractText()
        pagetext2  = pagetext2.strip()
        if 'Key Indicator' in pagetext and 'District' in pagetext and 'Key Indicator' in pagetext2 and 'Content' in pagetext2:
            pagetext = pagetext + pagetext2
        pdfFileObj.close()

        words      = re.split('\n', pagetext)
        words      = pagetext.split('\n')
        
        previndx = 1
        indx = []
        for i in range(1,100):
            for k in range(previndx, len(words)): 
                if str(i)+'.' in words[k]:
                    previndx = k 
                    indx.append(previndx)
                    break
        indx.append(len(words)-1)

        districts = [" ".join(words[t] for t in range(indx[i], indx[i+1])) for i in range(0, len(indx)-1) ]
        for i in range(1, len(districts)):
            if districts[i] == '':
                districts[i] = words[indx[i]]    
        districts = [districts[i].strip() for i in range(0, len(districts))]
        districts = [re.sub(' +', ' ', districts[i]) for i in range(0, len(districts))]
        districts = [districts[i].split(str(i+1)+'.')[1] for i in range(0, len(districts))]
        districts = [re.split('\d', districts[i])[0].strip() for i in range(0, len(districts))]
        districts = [re.sub(' - ', '-', districts[i]) for i in range(0, len(districts))]
        return districts
        

In [5]:
districts = {}
for key in states:
    districts[key] = getdistricts(key)

PDF for the state of AP exists
PDF for the state of AR exists
PDF for the state of AS exists
PDF for the state of BR exists
PDF for the state of CT exists
PDF for the state of GA exists
PDF for the state of GJ exists
PDF for the state of HR exists
PDF for the state of HP exists
PDF for the state of JH exists
PDF for the state of KA exists
PDF for the state of KL exists
PDF for the state of MP exists
PDF for the state of MH exists
PDF for the state of MN exists
PDF for the state of ML exists
PDF for the state of MZ exists
PDF for the state of NL exists
OR.pdf: No such file found

PDF for the state of PB exists
PDF for the state of RJ exists
PDF for the state of SK exists
PDF for the state of TN exists
PDF for the state of TG exists
PDF for the state of TR exists
UP.pdf: No such file found

PDF for the state of UT exists
PDF for the state of WB exists
PDF for the state of AN exists
PDF for the state of CH exists
PDF for the state of DD exists
PDF for the state of DL exists
PDF for the st

In [6]:
districts

{'Andhra_Pradesh': ['Anantapur',
  'Chittoor',
  'East Godavari',
  'Guntur',
  'Krishna',
  'Kurnool',
  'Prakasam',
  'Sri Potti Sriramulu Nellore',
  'Srikakulam',
  'Visakhapatnam',
  'Vizianagaram',
  'West Godavari',
  'Y.S.R.'],
 'Arunachal_Pradesh': ['Anjaw',
  'Changlang',
  'Dibang Valley',
  'East Kameng',
  'East Siang',
  'Kra Daadi',
  'Kurung Kumey',
  'Lohit',
  'Longding',
  'Lower Dibang Valley',
  'Lower Subansiri',
  'Namsai',
  'Papum Pare',
  'Siang',
  'Tawang',
  'Tirap',
  'Upper Siang',
  'Upper Subansiri',
  'West Kameng',
  'West Siang'],
 'Assam': ['Baksa',
  'Barpeta',
  'Biswanath',
  'Bongaigaon',
  'Cachar',
  'Charaideo',
  'Chirang',
  'Darrang',
  'Dhemaji',
  'Dhubri',
  'Dibrugarh',
  'Dima Hasao',
  'Goalpara',
  'Golaghat',
  'Hailakandi',
  'Hojai',
  'Jorhat',
  'Kamrup',
  'Kamrup',
  'Karbi Anglong',
  'Karimganj',
  'Kokrajhar',
  'Lakhimpur',
  'Majuli',
  'Morigaon',
  'Nagaon',
  'Nalbari',
  'Sivasagar',
  'Sonitpur',
  'South Salmara Ma

In [7]:
# CORRECT OR ADD ENTRIES TO DICTIONARY 'districts'
districts['Arunachal_Pradesh'][12] = 'Papumpare'

districts['Bihar'][7] = 'Buxar'

districts['Gujarat'][10] = 'Devbhumi Dwarka'
districts['Gujarat'][8] = 'Chhota Udaipur'

d = districts['Himachal_Pradesh']
districts['Himachal_Pradesh'] = [re.sub('&', 'and', d[i]) for i in range(0, len(d))]

districts['Karnataka'][1] = 'Bangalore Rural'

districts['Ladakh'][1] = 'Leh (Ladakh)'

d = districts['Madhya_Pradesh']
districts['Madhya_Pradesh'] = [re.sub('Narsimhapur', 'Narsinghpur', d[i]) for i in range(0, len(d))]

d = districts['Maharashtra']
districts['Maharashtra'][0] = 'Ahmednagar'
districts['Maharashtra'] = [re.sub('Buldana', 'Buldhana', d[i]) for i in range(0, len(d))]
districts['Maharashtra'][16] = 'Mumbai Suburban'
    
d = districts['Manipur']
districts['Manipur'] = [re.sub('Thoubal', 'Toubal', d[i]) for i in range(0, len(d))]

d = districts['Meghalaya']
districts['Meghalaya'] = [re.sub('East Jantia Hills', 'East Jaintia Hills', d[i]) for i in range(0, len(d))]

districts['Odisha'] = ['Anugul', 
                       'Balangir', 
                       'Baleshwar', 
                       'Bargarh', 
                       'Baudh', 
                       'Bhadrak', 
                       'Cuttack', 
                       'Debagarh', 
                       'Dhenkanal', 
                       'Gajapati', 
                       'Ganjam', 
                       'Jagatsinghapur', 
                       'Jajapur', 
                       'Jharsuguda', 
                       'Kalahandi',
                       'Kandhamal',
                       'Kendrapara',
                       'Kendujhar',
                       'Khordha',
                       'Koraput',
                       'Malkangiri',
                       'Mayurbhanj',
                       'Nabarangapur',
                       'Nayagarh',
                       'Nuapada',
                       'Puri',
                       'Rayagada',
                       'Sambalpur',
                       'Subarnapur',
                       'Sundargarh']

dP = districts['Punjab']
districts['Punjab'][7] = 'Gurudaspur'
districts['Punjab'] = [re.sub('Shahid Bhagat Singh Nagar', 'SBS Nagar',  dP[i]) for i in range(0, len(dP))]

dUT = districts['Uttarakhand']
districts['Uttarakhand'][5] = 'Haridwar'
districts['Uttarakhand'] = [re.sub('Udham Singh Nagar', 'Udam Singh Nagar',  dUT[i]) for i in range(0, len(dUT))]

districts['Uttar_Pradesh'] = ['Agra',
    'Aligarh',
    'Ambedkar Nagar',
    'Amethi',
    'Auraiya',
    'Azamgarh',
    'Baghpat',
    'Bahraich',
    'Ballia',
    'Balrampur',
    'Banda',
    'Barabanki',
    'Bareilly',
    'Basti',
    'Bijnor',
    'Budaun',
    'Bulandshahr',
    'Chandauli',
    'Chitrakoot',
    'Deoria',
    'Etah',
    'Etawah',
    'Faizabad',
    'Farrukhabad',
    'Fatehpur',
    'Firozabad',
    'Gautam Buddha Nagar',
    'Ghaziabad',
    'Ghazipur',
    'Gonda',
    'Gorakhpur',
    'Hamirpur',
    'Hapur',
    'Hardoi',
    'Jalaun',
    'Jaunpur',
    'Jhansi',
    'Jyotiba Phule Nagar',
    'Kannauj',
    'Kanpur Dehat',
    'Kanpur Nagar',
    'Kanshiram Nagar',
    'Kaushambi',
    'Kheri',
    'Kushinagar',
    'Lalitpur',
    'Lucknow',
    'Mahamaya Nagar',
    'Maharajganj',
    'Mahoba',
    'Mainpuri',
    'Mathura',
    'Mau',
    'Meerut',
    'Mirzapur',
    'Moradabad',
    'Muzaffarnagar',
    'Pilibhit',
    'Pratapgarh',
    'Prayagraj',
    'Rae Bareli',
    'Rampur',
    'Saharanpur',
    'Sambhal',
    'Sant Kabeer Nagar',
    'Sant Ravidas Nagar (Bhadohi)',
    'Shahjahanpur',
    'Shamli',
    'Shravasti',
    'Siddharthnagar',
    'Sitapur',
    'Sonbhadra',
    'Sultanpur',
    'Unnao',
    'Varanasi']

3. Save dictionaries to file using 'pickle' module

In [8]:
with open('states.pickle', 'wb') as f:
    # Pickle the 'states' dictionary using the highest protocol available.
    pickle.dump(states, f, pickle.HIGHEST_PROTOCOL)
    
with open('districts.pickle', 'wb') as f:
    # Pickle the 'districts' dictionary using the highest protocol available.
    pickle.dump(districts, f, pickle.HIGHEST_PROTOCOL)


4. Download district-level PDF files

In [9]:
def getdistrictdata(key):
    for d in districts[key]:
        path_to_file = states[key]+"_"+d+".pdf"
        url = "http://rchiips.org/nfhs/NFHS-5_FCTS/" + states[key] + "/" + d + ".pdf"
        if os.path.exists(path_to_file)==False and int(requests.get(url, stream=True).headers['Content-length']) > 0:
            response = requests.get(url)
            if response.status_code == 404:
                print(response.status_code, ' : ', url, ' does not exist.')
            if response.status_code == requests.codes.ok:
                file = open(path_to_file, "wb")
                file.write(response.content)
                file.close()

In [10]:
for key in states:
    getdistrictdata(key)  
print('Downloaded all files.')

Downloaded all files.
